## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-15-54-20 +0000,nyt,Tennessee House Special Election 2025: What to...,https://www.nytimes.com/2025/12/02/us/politics...
1,2025-12-02-15-53-17 +0000,nypost,Afghan national hit with federal charges of th...,https://nypost.com/2025/12/02/us-news/afghan-n...
2,2025-12-02-15-52-40 +0000,bbc,Nigeria's defence minister resigns amid kidnap...,https://www.bbc.com/news/articles/c62lg9047e0o...
3,2025-12-02-15-49-24 +0000,wapo,U.S. delegation ready to meet with Putin in la...,https://www.washingtonpost.com/world/2025/12/0...
4,2025-12-02-15-43-58 +0000,nyt,Sharks and Rays Gain Sweeping Protections from...,https://www.nytimes.com/2025/12/02/science/wil...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,63
305,new,22
1,house,18
117,was,12
136,out,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
204,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,146
171,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,132
263,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...,126
297,2025-12-01-19-00-50 +0000,nypost,Trump rails against ‘AOC of Tennessee’ in elec...,https://nypost.com/2025/12/01/us-news/presiden...,119
191,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
204,146,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...
45,62,2025-12-02-13-54-17 +0000,nypost,Pete Hegseth didn’t order killing of survivors...,https://nypost.com/2025/12/02/us-news/pete-heg...
36,52,2025-12-02-14-22-53 +0000,nypost,Mom of dead Texas cheerleader noticed ‘weird’ ...,https://nypost.com/2025/12/02/us-news/mom-of-d...
27,52,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...
171,51,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
40,44,2025-12-02-14-11-09 +0000,nyt,Thousands Greet Pope Leo as He Prays Near Site...,https://www.nytimes.com/2025/12/02/world/middl...
8,43,2025-12-02-15-34-11 +0000,nypost,Steve Witkoff greets Putin in Moscow for Russi...,https://nypost.com/2025/12/02/world-news/steve...
173,39,2025-12-02-03-13-14 +0000,nypost,Trump calls Tennessee special election a ‘big ...,https://nypost.com/2025/12/01/us-news/trump-ca...
30,37,2025-12-02-14-36-04 +0000,bbc,Reeves speech did not mislead on challenges fa...,https://www.bbc.com/news/articles/czj0ngnkl2vo...
76,36,2025-12-02-11-46-52 +0000,nypost,Deadly mass shooting in California erupted whe...,https://nypost.com/2025/12/02/us-news/californ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
